<a href="https://colab.research.google.com/github/lalit-jamdagnee/Kaggle-Data-Science-/blob/main/Digit_Classification_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'digit-recognizer:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3004%2F861823%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241004%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241004T105032Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D197c330f7e235876ed4e4c323aa0d7cc9021ec6f87e433bbb98bbf06c7f5cfabc0a2e7e90c7b0e2d49eec7ef27bc6db665422632d7e5a150971f057d15143ab2638bd0322b5025d7c553e44d15b18a24e1137049a5e346449bf2fcdcdf0ce8f10a25c83fc94ff6451b71f45143c1978e8d9f08228925cef94ed81883feaaa6cf410b5d38bffd1c31b33e258eaed4760d4d61139fff44a3174aa5a91c168ad1d0a9853e5aee9adeec3cab3041bd431dd38e28fd68af69f5b3c690c8293ebd819002fee459be56cf398165e3b63e122d27711d89d10b9446399315be505ccface280df75a3f1f812dbe1b88a2b2f036a3cd75d4696b13af9e66e3b6ac056bf929f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
df.head()

In [ ]:
# seperating features and labels from the training dataset

X = df.drop(['label'], axis=1)
y = df['label']

In [ ]:
# Scaling the dataset for better predictions

X = X/255.0
df_test = df_test/255.0

In [ ]:
# Let's see some images from the dataset
def plotting_digit(n):
    X_np = X.to_numpy()
    plt.figure(figsize=(6, 6))
    for i in range(n**2):
        plt.subplot(n, n, i+1)
        plt.imshow(X_np[i*10].reshape(28, 28), cmap="gray")
        plt.axis(False)
    plt.show()
plotting_digit(3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, stratify= y, random_state=42)
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)
print("y_train shape: :", y_train.shape)

In [ ]:
# Let's train our model for classifying  the test results

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)


In [ ]:
from sklearn.svm import SVC
svm_clf = SVC(C=1.0, kernel='rbf')
svm_clf.fit(X_train, y_train)

In [ ]:
# predicting the output on the test set created by us
y_pred_svm = svm_clf.predict(X_test)
y_pred_rf = rf_clf.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score

rf_accuracy = f1_score(y_test, y_pred_rf, average="macro")
svm_accuracy = f1_score(y_test, y_pred_svm, average="macro")
print("Accuracy of Random Forest Classifier: ", rf_accuracy)
print("Accuracy of SVM classifier: ", svm_accuracy)

In [ ]:
sample = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
sample.head()

In [ ]:
# Since our SVM classifier performs better than the RF classifier we will use it for the final
# prediction on the test dataset
predictions = svm_clf.predict(df_test)

In [ ]:
# Creating the output dataframe as per the specifications of the submission file
output = pd.DataFrame(predictions, columns=['Label'])
output['ImageId'] = output.index + 1
output = output[['ImageId', 'Label']]
output.head()

In [ ]:
output.to_csv('submission.csv', index=False)